In [1]:
import pandas as pd
        
pure_isw = pd.read_csv('pure_isw.csv')
pure_isw

,text_for_vect2,date
0,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-25
1,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-26
2,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-27
3,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-28
4,"[ ' frederick ' , ' kagan ' , ' george ' , ' b...",2022-03-01
...,...,...
324,"[ ' karolina ' , ' hird ' , ' grace ' , ' mapp...",2023-01-21
325,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-22
326,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-23
327,"[ ' karolina ' , ' hird ' , ' riley ' , ' bail...",2023-01-24


In [2]:
import gensim
from gensim.utils import simple_preprocess

# Tokenize the strings
tokenized_text = [simple_preprocess(text) for text in pure_isw["text_for_vect2"]]

# Create a dictionary
dic = gensim.corpora.Dictionary(tokenized_text)

# Create a bag of words corpus
bow_corpus = [dic.doc2bow(doc) for doc in tokenized_text]

# Train LDA model
lda = gensim.models.LdaMulticore(bow_corpus, id2word=dic, num_topics=10, passes=15, workers=4)




In [3]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, bow_corpus, dictionary=dic)
vis 

/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=5055) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.159960  0.037210       1        1  42.641082
8      0.171497  0.005513       2        1  42.150297
7      0.246066 -0.136655       3        1   8.672752
0      0.094913 -0.039137       4        1   2.527992
2      0.136394  0.083606       5        1   2.477725
3      0.079511  0.102430       6        1   1.438348
9     -0.166704 -0.024992       7        1   0.073866
1     -0.235669 -0.009341       8        1   0.005980
4     -0.242681 -0.009550       9        1   0.005980
5     -0.243288 -0.009084      10        1   0.005980, topic_info=          Term          Freq         Total Category  logprob  loglift
312    russian  31413.000000  31413.000000  Default  30.0000  30.0000
140      force  16671.000000  16671.000000  Default  29.0000  29.0000
369  ukrainian  16111.000000  16111.000000  Default  28.0000  28.0000
368    ukraine   5368.000000   5368.000000  Default  27.0000  27.0000
241     oblast   7374.000000   7374.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
311     russia      0.004355   3029.745578  Topic10  -8.9309  -3.7281
4      advance      0.004256   1918.182814  Topic10  -8.9540  -3.2941
415     attack      0.004237   2437.710845  Topic10  -8.9584  -3.5382
503    kremlin      0.004055   1510.755048  Topic10  -9.0024  -3.1037
362      troop      0.004102   1954.746536  Topic10  -8.9909  -3.3498

[785 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
7390      1  0.238086   abbreviate
7390      2  0.079362   abbreviate
7390      4  0.555534   abbreviate
5479      2  0.455571     adamivka
5479      5  0.520653     adamivka
...     ...       ...          ...
2160      3  0.958300  zaporizhiya
1019      3  0.985786   zaporizhya
948       3  0.808374    zaprozhya
3658      2  0.983393       zolote
5556      5  0.842721     zvanivka

[1378 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 8, 1, 3, 4, 10, 2, 5, 6])

In [4]:
def remove_smth(text):
    stops = ["yet", "zaporizhzhia", "we", "y",]
    words = text.split()
    filtered_words = [w for w in words if w not in stops]
    return " ".join(filtered_words)

pure['text_for_vect2'] = pure["text_for_vect2"].apply(remove_smth)

NameError: name 'pure' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_features=700)
tfidf_matrix = tfidf_vect.fit_transform(pure['text_for_vect2'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vect.get_feature_names_out())
vect_df = pd.concat([pure["date"], tfidf_df], axis=1)

In [ ]:

from wordcloud import WordCloud

w = WordCloud()
custom_stopwords = ["yet", "zaporizhzhia", "we", "y",]
stopwords = set(list(w.stopwords) + custom_stopwords)

import matplotlib.pyplot as plt
x = tfidf_vect.vocabulary_
cloud = WordCloud(stopwords=stopwords).generate_from_frequencies(x)

plt.imshow(cloud )
plt.axis("off")
plt.show()

